In [ ]:
"""
A comprehensive demonstration of all three pipeline execution modes:
1. `run()`: For direct, one-shot execution.
2. `step_by_step()` with a `for` loop: For observing a full run.
3. `step_by_step()` with manual `next()`: For interactive debugging.
4. `start_execution()`: For advanced, stateful, chainable execution.
"""

import pandas as pd
from axisfuzzy.analysis.pipeline import FuzzyPipeline
from axisfuzzy.analysis.components.basic import (
    NormalizationTool, StatisticsTool, SimpleAggregationTool)

In [ ]:
# --- 1. Setup: Create Data and Pipeline ---
data = pd.DataFrame({
    'Cost': [100, 200, 150, 180],
    'Quality': [8, 9, 7, 8.5],
    'Speed': [50, 60, 55, 58]
}, index=['Product_A', 'Product_B', 'Product_C', 'Product_D'])

p = FuzzyPipeline()
data_input = p.input()
normalized = p.add(NormalizationTool(method='min_max', axis=0).run, data=data_input)
stats = p.add(StatisticsTool(axis=0).run, data=normalized)
aggregated = p.add(SimpleAggregationTool(operation='mean', axis=1).run, data=normalized)

print("=" * 70)
print("Pipeline and Data are ready for demonstration.")
print("=" * 70)

In [ ]:
# --- 2. Mode 1: `run()` - The Direct Approach ---
print("\n>>> MODE 1: Direct execution with `run()` <<<")
final_result, intermediate_results = p.run({"default_input": data}, return_intermediate=True)
print("✅ `run()` execution complete.")
print("Final Result (output of terminal nodes):")
print(final_result)
print(f"Total intermediate steps captured: {len(intermediate_results)}")
print("-" * 70)

In [ ]:
# --- 3. Mode 2: `step_by_step()` with `for` loop - The Observer's Approach ---
print("\n>>> MODE 2: Iterative execution with `step_by_step()` and `for` loop <<<")
for step_result in p.step_by_step({'default_input': data}):
    print(f"  - Executed: {step_result['step_name']} (Index: {step_result['step_index']})")
print("✅ `for` loop execution complete.")
print("-" * 70)

In [ ]:
# --- 4. Mode 3: `step_by_step()` with manual `next()` - The Debugger's Approach ---
print("\n>>> MODE 3: Manual step-by-step with `step_by_step()` and `next()` <<<")
iterator = p.step_by_step({'default_input': data})
try:
    print("Press Enter to execute each step manually.")

    input("  Press Enter for Step 1...")
    step1 = next(iterator)
    print(f"  ✅ Step 1 ('{step1['step_name']}') Done. Result keys: {list(step1['result'].keys())}")

    input("  Press Enter for Step 2...")
    step2 = next(iterator)
    print(f"  ✅ Step 2 ('{step2['step_name']}') Done. Stats: {step2['result']['statistics']['mean']:.2f} (mean)")

    input("  Press Enter for Step 3...")
    step3 = next(iterator)
    print(f"  ✅ Step 3 ('{step3['step_name']}') Done. Aggregated shape: {step3['result']['aggregated_values'].shape}")

except StopIteration:
    pass
finally:
    print("✅ Manual `next()` execution demonstration complete.")
print("-" * 70)

In [ ]:
# --- 5. Mode 4: `start_execution()` - The Advanced/Chainable Approach ---
print("\n>>> MODE 4: Chainable execution with `start_execution()` <<<")
try:
    state0 = p.start_execution({"default_input": data})
    print(f"  Initial state created: {state0}")

    state1 = state0.run_next()
    print(f"  State after step 1: {state1}")

    state2 = state1.run_next()
    print(f"  State after step 2: {state2}")

    final_state = state2.run_all()
    print(f"  Final state after running all remaining: {final_state}")

    print("\n  Final state contains results for all steps:")
    for step_id in final_state.step_results:
        step_name = final_state.pipeline.get_step_info(step_id)['display_name']
        print(f"    - {step_name}")

except Exception as e:
    print(f"  An error occurred: {e}")
finally:
    print("✅ Chainable execution demonstration complete.")
print("=" * 70)